In [0]:

/*
show create table hive_metastore.default.widetable_5g_cell_hr_tmp1

CREATE TABLE apachedataingestion.nr.widetable_5g_cell_hr_qci_gold2(
    CalendarDate DATE,
    Hour24 STRING,
    GNODEB_ID STRING,
    du_id STRING,
    GCELL_ID BIGINT,
    qci STRING,
    MARKET_NAME STRING,
    NR_TYPE STRING,
    SUB_MARKET_ID STRING,
    SUB_MARKET_NAME STRING,  
    S5NR_DLUserPerceivedTput_Mbps_Num DOUBLE,
    S5NR_DLUserPerceivedTput_Mbps_DENOM DOUBLE,
    S5NR_TotalERABSetupFailureRate_PCT_Num DOUBLE,
    S5NR_TotalERABSetupFailureRate_PCT_Denom DOUBLE,
    S5NR_TotalDLRLCLayerDataVolume_MB DOUBLE,
    S5NR_TotalULRLCLayerDataVolume_MB DOUBLE,
    S5NR_DLRLCReTx_PCT_Num DOUBLE,
    S5NR_DLRLCReTx_PCT_Denom DOUBLE,
    S5NR_RLC_Delay_ms_Num DOUBLE,
    S5NR_RLC_Delay_ms_Denom DOUBLE,
    S5NR_AirIF_Delay_ms_Num DOUBLE,
    S5NR_AirIF_Delay_ms_Denom DOUBLE)
USING delta
PARTITIONED BY (CalendarDate)
TBLPROPERTIES (
  'delta.checkpoint.writeStatsAsJson' = 'false',
  'delta.checkpoint.writeStatsAsStruct' = 'true',
  'delta.columnMapping.mode' = 'name',
  'delta.minReaderVersion' = '2',
  'delta.minWriterVersion' = '5')

select * from apachedataingestion.nr.widetable_5g_cell_hr_qci_gold2 

*/

delete from apachedataingestion.nr.widetable_5g_cell_hr_qci_gold2 
where CalendarDate >= date_format(date_add(now(), {{ days }}),"yyyy-MM-dd") and CalendarDate < date_format(date_add(now(), ({{ days }})+1),"yyyy-MM-dd") 
;

insert into apachedataingestion.nr.widetable_5g_cell_hr_qci_gold2
with conf_du (          
			          	select substring(DU_ID,1,10) GNODEB_ID,DU_ID,GCELL_ID,DU_NAME,MARKET_NAME,NETWORK_NAME,SUB_MARKET_ID,SUB_MARKET_NAME,cast (replace(substring(DU_ID,1,10),'VZ_','') as bigint) kGNODEB_ID ,cast (replace(DU_ID,'VZ_','') as bigint) kDU_ID
			          	-- select count(distinct concat(GNODEB_ID,'_',DU_ID,'_',GCELL_ID)) --514527
                  		from hive_metastore.`default`.vz_conf_gnode_du_gcell_silver  where not(length(GNODEB_ID) < 10 or length(du_id) < 14) 
				  	union all
						select a.* from (
							select concat('VZ_',gNB_ID) GNODEB_ID,concat('VZ_',DU_ID) DU_ID,CellIdentity GCELL_ID,'' DU_NAME,'' MARKET_NAME,'' NETWORK_NAME,'' SUB_MARKET_ID,'' SUB_MARKET_NAME,cast (gNB_ID as bigint) kGNODEB_ID ,cast (DU_ID as bigint) kDU_ID
							-- select count(*)
							from hive_metastore.`default`.jwk_mapping_live 
						) a
						full join hive_metastore.`default`.vz_conf_gnode_du_gcell_silver b on a.kDU_ID=cast (replace(b.DU_ID,'VZ_','') as bigint) and a.GCELL_ID=b.GCELL_ID
						where b.DU_ID is null 
)
select 
CalendarDate ,
Hour24 ,
GNODEB_ID,
du_id ,
GCELL_ID ,
qci,
MARKET_NAME ,
case when length(du_id)=14 then 
	case when substring(du_id, 7, 1)=0 then 'FR2' 	 when substring(du_id, 7, 1)=9 then 
				case when substring(du_id, 11, 1)=2 then 'FR1/LS3' when substring(du_id, 11, 1)=3 then 'FR1/LS6' 
				end
	end
else '' end NR_TYPE,
substring(du_id,4,3) SUB_MARKET_ID,
SUB_MARKET_NAME ,
max(S5NR_DLUserPerceivedTput_Mbps_Num) S5NR_DLUserPerceivedTput_Mbps_Num,
max(S5NR_DLUserPerceivedTput_Mbps_DENOM) S5NR_DLUserPerceivedTput_Mbps_DENOM,
max(S5NR_TotalERABSetupFailureRate_PCT_Num) S5NR_TotalERABSetupFailureRate_PCT_Num,
max(S5NR_TotalERABSetupFailureRate_PCT_Denom) S5NR_TotalERABSetupFailureRate_PCT_Denom,
max(S5NR_TotalDLRLCLayerDataVolume_MB) S5NR_TotalDLRLCLayerDataVolume_MB,
max(S5NR_TotalULRLCLayerDataVolume_MB) S5NR_TotalULRLCLayerDataVolume_MB,
max(S5NR_DLRLCReTx_PCT_Num) S5NR_DLRLCReTx_PCT_Num,
max(S5NR_DLRLCReTx_PCT_Denom) S5NR_DLRLCReTx_PCT_Denom,
max(S5NR_RLC_Delay_ms_Num) S5NR_RLC_Delay_ms_Num,
max(S5NR_RLC_Delay_ms_Denom) S5NR_RLC_Delay_ms_Denom,
max(S5NR_AirIF_Delay_ms_Num) S5NR_AirIF_Delay_ms_Num,
max(S5NR_AirIF_Delay_ms_Denom) S5NR_AirIF_Delay_ms_Denom
from (
			select 
			COALESCE(M0.CalendarDate,T1.CalendarDate,T2.CalendarDate,T3.CalendarDate) CalendarDate,
			COALESCE(M0.Hour24,T1.Hour24,T2.Hour24,T3.Hour24) Hour24,
			COALESCE(M0.GNODEB_ID,T1.GNODEB_ID,T2.GNODEB_ID,T3.GNODEB_ID) GNODEB_ID,
			COALESCE(M0.du_id,T1.du_id,T2.du_id,T3.du_id) du_id,
			COALESCE(M0.GCELL_ID,T1.GCELL_ID,T2.GCELL_ID,T3.GCELL_ID) GCELL_ID,
            COALESCE(M0.qci,T1.qci,T2.qci,T3.qci) qci,
			COALESCE(M0.MARKET_NAME,T1.MARKET_NAME,T2.MARKET_NAME,T3.MARKET_NAME) MARKET_NAME,
			COALESCE(M0.SUB_MARKET_NAME,T1.SUB_MARKET_NAME,T2.SUB_MARKET_NAME,T3.SUB_MARKET_NAME) SUB_MARKET_NAME,
			(8 * DLUeThruVol/1000) S5NR_DLUserPerceivedTput_Mbps_NUM,
			(DLUeThruTime/  power(10,6)) S5NR_DLUserPerceivedTput_Mbps_DENOM,
			ErabAddAtt-ErabAddSucc S5NR_TotalERABSetupFailureRate_PCT_Num, 
			ErabAddAtt  S5NR_TotalERABSetupFailureRate_PCT_Denom,			
			RlcDLByte/1000 S5NR_TotalDLRLCLayerDataVolume_MB,
			RlcULByte/1000 S5NR_TotalULRLCLayerDataVolume_MB,
			RlcDLRePktCnt	S5NR_DLRLCReTx_PCT_Num,
			RlcDLPktCnt S5NR_DLRLCReTx_PCT_Denom,		
            RlcSduDelayTot/1000		S5NR_RLC_Delay_ms_Num,
			RlcSduDelayCnt			S5NR_RLC_Delay_ms_Denom,
			AirIfDelayTot/1000		S5NR_AirIF_Delay_ms_Num,
			AirIfDelayCnt			S5NR_AirIF_Delay_ms_Denom
			from (
					select Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,b.GCELL_ID,split(a.LOCATION,'/')[1]  qci,MARKET_NAME,SUB_MARKET_NAME,
							SUM(DLUeThruVol_Kbytes_SUM) DLUeThruVol,
							SUM(DLUeThruTime_us_SUM) DLUeThruTime
							-- select *
						from  apachedataingestion.nr.downlinkuethroughput_nr_hr_silver a
						inner join conf_du b on concat('VZ_',left(REGEXP_REPLACE(NE_NAME, '[^0-9]', ''),11))=b.DU_ID and replace(split(a.LOCATION,'/')[0],'CellIdentity','')=b.gcell_id
						WHERE 5=5 and  Daily  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd") 
                        and split(a.LOCATION,'/')[1] in ('QCI7','QCI8','QCI9','QCI131','QCI132','QCI133')
						GROUP BY Daily,Hourly,b.du_id,GNODEB_ID,b.GCELL_ID,MARKET_NAME,SUB_MARKET_NAME,split(a.LOCATION,'/')[1]
			) M0 
            full join  (
					select CalendarDate,Hour24,du_id,GNODEB_ID,GCELL_ID,qci,MARKET_NAME,SUB_MARKET_NAME,
						SUM(ErabAddAtt) ErabAddAtt, 
						SUM(ErabAddSucc) ErabAddSucc 
					from (
								SELECT Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,b.GCELL_ID,split(a.LOCATION,'/')[2]  qci,MARKET_NAME,SUB_MARKET_NAME,
								SUM(ErabAddAtt_count_SUM) ErabAddAtt,
								SUM(ErabAddSucc_count_SUM) ErabAddSucc
                                -- select *
								from apachedataingestion.nr.erabadditionpergnb_hr_silver a
								inner join conf_du b on cast(replace(split(a.LOCATION,'/')[0],'gNB_ID','') as bigint)=b.kGNODEB_ID and replace(split(a.LOCATION,'/')[1],'CellIdentity','')=b.gcell_id
								WHERE 5=5 and  Daily  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd") 
                                and split(a.LOCATION,'/')[2] in ('QCI7','QCI8','QCI9','QCI131','QCI132','QCI133')
								GROUP BY Daily,Hourly,b.du_id,GNODEB_ID,b.GCELL_ID,MARKET_NAME,SUB_MARKET_NAME,split(a.LOCATION,'/')[2]
								union all
								SELECT Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,b.GCELL_ID,split(a.LOCATION,'/')[3]  qci,MARKET_NAME,SUB_MARKET_NAME,
								SUM(ErabAddAtt_count_SUM) ErabAddAtt,
								SUM(ErabAddSucc_count_SUM) ErabAddSucc
                                -- select *
								from apachedataingestion.nr.erabadditionpergnbwithduidentifier_hr_silver a
								inner join conf_du b on cast(replace(split(a.LOCATION,'/')[0],'gNB_ID','') as bigint)=b.kGNODEB_ID and replace(split(a.LOCATION,'/')[2],'CellIdentity','')=b.gcell_id
								WHERE 5=5 and  Daily  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd") 
                                and split(a.LOCATION,'/')[3] in ('QCI7','QCI8','QCI9','QCI131','QCI132','QCI133')
								GROUP BY Daily,Hourly,b.du_id,GNODEB_ID,b.GCELL_ID,MARKET_NAME,SUB_MARKET_NAME,split(a.LOCATION,'/')[3]
					) x GROUP BY CalendarDate,Hour24,du_id,GNODEB_ID,GCELL_ID,MARKET_NAME,SUB_MARKET_NAME,qci 
			) T1 on M0.CalendarDate=T1.CalendarDate and M0.Hour24=T1.Hour24 and M0.du_id=T1.du_id and M0.GCELL_ID=T1.GCELL_ID  and M0.qci=T1.qci                  
			full join  (
				SELECT Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,b.GCELL_ID,split(a.LOCATION,'/')[1]  qci,MARKET_NAME,SUB_MARKET_NAME,
						SUM(RlcDLByte_Kbytes_SUM) RlcDLByte,
						SUM(RlcULByte_Kbytes_SUM) RlcULByte,
						SUM(RlcDLRePktCnt_count_SUM) RlcDLRePktCnt,
						SUM(RlcDLPktCnt_count_SUM) RlcDLPktCnt
						-- select *
				from apachedataingestion.nr.rlcpacket_hr_silver a
				inner join conf_du b on concat('VZ_',left(REGEXP_REPLACE(NE_NAME, '[^0-9]', ''),11))=b.DU_ID and replace(split(a.LOCATION,'/')[0],'CellIdentity','')=b.gcell_id
				WHERE 5=5 and  Daily  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd") 
                and split(a.LOCATION,'/')[1] in ('QCI7','QCI8','QCI9','QCI131','QCI132','QCI133')
				GROUP BY Daily,Hourly,b.du_id,GNODEB_ID,b.GCELL_ID,MARKET_NAME,SUB_MARKET_NAME,split(a.LOCATION,'/')[1]
			) T2 on M0.CalendarDate=T2.CalendarDate and M0.Hour24=T2.Hour24 and M0.du_id=T2.du_id and M0.GCELL_ID=T2.GCELL_ID 	and M0.qci=T2.qci         
            full join  (
				SELECT  Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,b.GCELL_ID,split(a.LOCATION,'/')[1]  qci,MARKET_NAME, SUB_MARKET_NAME, 
                    sum(RlcSduDelayTot_us_SUM) RlcSduDelayTot,
					sum(RlcSduDelayCnt_count_SUM) RlcSduDelayCnt,
					sum(AirIfDelayTot_us_SUM) AirIfDelayTot,
					sum(AirIfDelayCnt_count_SUM) AirIfDelayCnt
                    -- select *
                from apachedataingestion.nr.airinterfacedelayfornsa_nr_hr_silver    a
                inner join conf_du b on concat('VZ_',left(REGEXP_REPLACE(NE_NAME, '[^0-9]', ''),11))=b.DU_ID and replace(split(a.LOCATION,'/')[0],'CellIdentity','')=b.gcell_id
                    WHERE 5=5 and  Daily  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd") 
                    and split(a.LOCATION,'/')[1] in ('QCI7','QCI8','QCI9','QCI131','QCI132','QCI133')
				GROUP BY Daily,Hourly,b.du_id,GNODEB_ID,b.GCELL_ID,MARKET_NAME,SUB_MARKET_NAME,split(a.LOCATION,'/')[1]
			) T3 on M0.CalendarDate=T3.CalendarDate and M0.Hour24=T3.Hour24 and M0.du_id=T3.du_id and M0.GCELL_ID=T3.GCELL_ID  and M0.qci=T3.qci                    
) X
GROUP BY CalendarDate,Hour24,GNODEB_ID,du_id,GCELL_ID,qci,MARKET_NAME,SUB_MARKET_NAME;







